# **Reference**

# 1.my pa1 source code
# 2.https://www.kaggle.com/aninditapani/will-it-rain-tomorrow

# **DATA PREPROCESSING**

In [ ]:
#import necessary libraries
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from collections import Counter
import io
import matplotlib.pyplot as plt
import scipy.stats
import scipy.special
from sklearn import metrics
from sklearn import linear_model
import sklearn.linear_model
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import time
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import Imputer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import seaborn as sns

#import data file
df = pd.read_csv('weatherAUS.csv')
print('Size of weather data frame is :',df.shape)
#Drop unnecessary column Date, Location, and RISK_MM which may reveal the answer to the model that will reduce the predictability
#location dropped because we are predicting whether it will rain in Austrailia
df = df.drop(columns=['RISK_MM','Date','Location'],axis=1)
#replace string categories into integer values
def replaceVals(column_name):
    list1 = ['W','WNW','WSW','NE','NNW','N','NNE','SW','ENE','SSE','S','NW','SE','ESE','E','SSW']
    inta = 0
    for a in list1:
        df[column_name] = df[column_name].replace(a,inta)
        inta = inta+1
replaceVals('WindGustDir')
replaceVals('WindDir9am')
replaceVals('WindDir3pm')
#replace empty slots with NaN and convert No with 0 and Yes with 1
df = df.replace(0, np.NaN)
df['RainToday'].replace({'No':  int(0), 'Yes': int(1)},inplace = True)
df['RainTomorrow'].replace({'No': int(0), 'Yes': int(1)},inplace = True)

df[0:10]

In [ ]:
#imputation of dataset with imputer
fill_NaN = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
imputed_df = pd.DataFrame(fill_NaN.fit_transform(df))
imputed_df.columns = df.columns
imputed_df.index = df.index
imputed_df[0:10]
df = imputed_df
df[0:10]

In [ ]:
#reference:https://medium.com/ibm-data-science-experience/missing-data-conundrum-exploration-and-imputation-techniques-9f40abe0fd87
#standardize the values in the dataset
scaler = preprocessing.MinMaxScaler()
scaler.fit(df)
df = pd.DataFrame(scaler.transform(df), index=df.index, columns=df.columns)
X = df.loc[:,df.columns!='RainTomorrow']
y = df[['RainTomorrow']]
train_X,test_X,train_y,test_y = train_test_split(X,y,test_size = 0.2,random_state=42)
df.iloc[0:10]

# **Models**

# **Logistic Regression**

In [ ]:
def plot_grid_searchlr(cv_results, grid_param, name_param):
    # Get Test Scores Mean and std for each grid search
    scores_mean = cv_results['mean_test_score']
    scores_sd = cv_results['std_test_score']

    # Plot Grid search scores
    _, ax = plt.subplots(1,1)

    lst = []
    # Param1 is the X-axis, Param 2 is represented as a different curve (color line)
    for cnt in range(len(grid_param)):
        s = str(grid_param[cnt])
        lst.append(s)
        print("Setting", cnt, ': ', s, 'test score:', round(scores_mean[cnt],3))
        
    ax.plot([1,2,3,4,5,6], scores_mean, '-o', label= 'test score')

    ax.set_title("Grid Search Scores", fontsize=20, fontweight='bold')
    ax.set_xlabel(name_param, fontsize=16)
    ax.set_ylabel('CV Average Score', fontsize=16)
    ax.legend(loc="best", fontsize=15)

    ax.grid('off')

#function used to plot learning curve
#function to draw graph
def plot_log_reg(x, y, title, clf, xmin=None, xmax=None, alpha=1, ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    else:
        fig = ax.figure
    ax.set_title(title)
    ax.scatter(x, y, color='black', zorder=20, alpha=alpha)
    if xmin is None:
        xmin = x.min()
    if xmax is None:
        xmax = x.max()
    X_test = np.linspace(xmin, xmax,300)
    loss = scipy.special.expit(X_test * clf.coef_[0][0] + clf.intercept_).ravel()
    ax.plot(X_test, loss, linewidth=3)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    fig.tight_layout()
    return fig, ax

#function to draw confusion matrix
def draw_confusion(test, pred,title):
    cm = metrics.confusion_matrix(test, pred)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    # drawing confusion matrix using matplotlib
    plt.figure(figsize=(9,9))
    plt.imshow(cm, interpolation='nearest', cmap='Pastel1')
    plt.title(title, size = 15)
    plt.colorbar()
    tick_marks = np.arange(2)
    plt.xticks(tick_marks, ["0", "1"], rotation=45, size = 10)
    plt.yticks(tick_marks, ["0", "1"], size = 10)
    plt.tight_layout()
    plt.ylabel('Actual label', size = 10)
    plt.xlabel('Predicted label', size = 10)
    width, height = cm.shape
    for x in range(width):
        for y in range(height):
            plt.annotate(str(cm[x][y]), xy=(y, x), 
            horizontalalignment='center',
            verticalalignment='center')

#function to do logistic regression
def logistic_regression():
    print("RESULT ON LOGISTIC REGRESSION")
    print()
    print()
    parameters = {'penalty':('l1','l2'), 'eta0':[0.01, 0.05, 0.1]}
    lgr = sklearn.linear_model.SGDClassifier()
    clf_lgr = GridSearchCV(lgr, parameters,cv = 5)
    clf_lgr.fit(train_X, train_y)
    print('Best parameter is:')
    print(clf_lgr.best_params_)
    plot_grid_searchlr(clf_lgr.cv_results_, clf_lgr.cv_results_['params'], 'N Estimators')
    penaltys = clf_lgr.best_params_['penalty']
    eta0s = clf_lgr.best_params_['eta0']
    t0 = time.time()
    clf_sgd = sklearn.linear_model.SGDClassifier(penalty = penaltys, random_state = None, loss = 'log', max_iter=50, tol=1e-3, verbose=1,eta0=eta0s,early_stopping = False)
    print("Model Setting:")
    print(clf_sgd.get_params([True]))
    print()
    
    #Fit training data on each model
    print("Fitting model")
    clf_sgd.fit(train_X,train_y)
    print()
    
    #print intercept and coefficents of models
    print('model_intercept:',clf_sgd.intercept_)
    print('model_coef:', clf_sgd.coef_)
    
    #Comparing Mean Accuracy of model on training and testing sets
    score_train = clf_sgd.score(train_X,train_y)
    score_test = clf_sgd.score(test_X,test_y)
    print("Mean Accuracy of training set is {0} and test set is {1}  ".format(score_train, score_test))
    print()
    
    #Drawing confusion matrix to visualize and compare on both training and test set
    prediction_train = clf_sgd.predict(train_X)
    prediction_test = clf_sgd.predict(test_X)
    
    draw_confusion(train_y,prediction_train,"Confusion Matrix of train set")
    draw_confusion(test_y,prediction_test,"Confusion Matrix of test set")
    print("time taken is:")
    print(time.time()-t0)
    return clf_sgd
clf_sgd = logistic_regression()


In [ ]:
#to visualize logistic regression
sns.regplot(x='Rainfall', y='RainTomorrow', data=df, logistic=True)

# **Neural Network**

In [ ]:
# Visualize the results of hyperparameter tuning
def plot_grid_search(cv_results, grid_param, name_param):
    # Get Test Scores Mean and std for each grid search
    scores_mean = cv_results['mean_test_score']
    scores_sd = cv_results['std_test_score']

    # Plot Grid search scores
    _, ax = plt.subplots(1,1)

    lst = []
    # Param1 is the X-axis, Param 2 is represented as a different curve (color line)
    for cnt in range(len(grid_param)):
        s = str(grid_param[cnt])
        lst.append(s)
        print("Setting", cnt, ': ', s, 'test score:', round(scores_mean[cnt],3))
        
    ax.plot([1,2,3,4,5,6,7,8,9,10], scores_mean, '-o', label= 'test score')

    ax.set_title("Grid Search Scores", fontsize=20, fontweight='bold')
    ax.set_xlabel(name_param, fontsize=16)
    ax.set_ylabel('CV Average Score', fontsize=16)
    ax.legend(loc="best", fontsize=15)

    ax.grid('off')

#function used to plot learning curve
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

def neural_network(trainx,trainy,testx,testy,title):
    #10 hyperparameters and going to choose best one out of them
    tuned_parameters = [{'hidden_layer_sizes':[(1, )], 'learning_rate':['constant'], 'learning_rate_init':[0.01]},
                        {'hidden_layer_sizes':[(2, )],'learning_rate':['constant'], 'learning_rate_init':[0.01]},
                        {'hidden_layer_sizes':[(3, )],'learning_rate':['constant'], 'learning_rate_init':[0.01]},
                        {'hidden_layer_sizes':[(4, )],'learning_rate':['constant'], 'learning_rate_init':[0.01]},
                        {'hidden_layer_sizes':[(5, )],'learning_rate':['constant'], 'learning_rate_init':[0.01]},
                        {'hidden_layer_sizes':[(6, )],'learning_rate':['constant'], 'learning_rate_init':[0.01]},
                        {'hidden_layer_sizes':[(7, )],'learning_rate':['constant'], 'learning_rate_init':[0.01]},
                        {'hidden_layer_sizes':[(8, )],'learning_rate':['constant'], 'learning_rate_init':[0.01]},
                        {'hidden_layer_sizes':[(9, )],'learning_rate':['constant'], 'learning_rate_init':[0.01]},
                        {'hidden_layer_sizes':[(10, )],'learning_rate':['constant'], 'learning_rate_init':[0.01]}]
    
    scores = ['precision', 'recall']
    #cross validation performed to determine the best hyperparameter
    for score in scores:
        print("# Tuning hyper-parameters for %s" % score)
        print()
        
        mlp =MLPClassifier(max_iter=300, alpha=1e-4, solver='sgd', verbose=True, tol=1e-4, random_state=1,early_stopping = True)
        clf = GridSearchCV(mlp, tuned_parameters, cv=5, scoring='%s_macro' % score)
        clf.fit(trainx, trainy)
        print("Best parameters set found on development set:")
        print()
        print(clf.best_params_)

        print()
        print("Grid scores on development set:")
        print()
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))
        print()
        print("Detailed classification report:")
        print()
        print("The model is trained on the full development set.")
        print("The scores are computed on the full evaluation set.")
        print()
        y_true, y_pred = testy, clf.predict(testx)
        print(classification_report(y_true, y_pred))
        print()
    
    #plot the cross validation result    
    plot_grid_search(clf.cv_results_, clf.cv_results_['params'], 'N Estimators')
    
    #train the model with full training set and show the best loss score before overfitting happens
    best_para = clf.best_params_['hidden_layer_sizes']
    t0 = time.time()
    mlp =MLPClassifier(hidden_layer_sizes=best_para, learning_rate='constant', learning_rate_init=0.001, max_iter=300, alpha=1e-4, solver='sgd', verbose=True, tol=1e-4, random_state=1,early_stopping = True)
    
    #to get the model setting
    print()
    print("Parameter setting for MLPClassifier {0}".format(title))
    print()
    print(mlp.get_params([True]))
    print()
    
    #to calculate best loss on test set
    mlp.fit(testx, testy)
    testloss = mlp.loss_
    
    #to train the model on training set and calculate train loss
    mlp.fit(trainx,trainy)
    print()
    print("Best loss before overfitting for train set is {0}".format(mlp.loss_))
    print("Best loss before overfitting for test set is {0}".format(testloss))
    
    #Classification of training and test set visualized in mean accuracy and confusion matrix
    print()
    print("Experiment results on {0} data set ".format(title))
    train_score = mlp.score(trainx,trainy)
    test_score = mlp.score(testx,testy)
    print()
    print("Mean accuracy score of {0} data training set is {1} and test set is {2}".format(title,train_score,test_score))
    train_predict = mlp.predict(trainx)
    test_predict = mlp.predict(testx)
    draw_confusion(trainy,train_predict,"Confusion Matrix of {0} train set".format(title))
    draw_confusion(testy,test_predict,"Confusion Matrix of {0} test set".format(title))
    print("time taken is:")
    print(time.time()-t0)
    '''
    #plot the learning curve to show performance change over time
    title = "Learning Curves (Naive Bayes)"
    cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)
    estimator = GaussianNB()
    plot_learning_curve(mlp, title, trainx, trainy, ylim=(0.1, 1.01), cv=cv, n_jobs=4)
    title = "Learning Curves (SVM, RBF kernel, $\gamma=0.001$)"
    cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
    estimator = SVC(gamma=0.001)
    plot_learning_curve(mlp, title, trainx, trainy, (0.1, 1.01), cv=cv, n_jobs=4)
    plt.show()
    '''
    return mlp
mlp = neural_network(train_X,train_y,test_X,test_y,'austrailia')


# **SVM**

In [ ]:
from sklearn.svm import SVC
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
#to find the best parameter setting
parameters = {'kernel':('linear', 'rbf','poly'), 'C':[1, 10]}
svr = SVC()
clf_svm = GridSearchCV(svr, parameters,cv = 5)
clf_svm.fit(train_X, train_y)
print('Best parameter is:')
print(clf_svm.best_params_)

In [ ]:
#to plot the result of parameter setting and fit the model with full training data 
plot_grid_searchlr(clf_svm.cv_results_, clf_svm.cv_results_['params'], 'N Estimators')
Cs = clf_svm.best_params_['C']
kernels = clf_svm.best_params_['kernel']
svr = SVC(C = Cs,kernel = kernels)
t0 = time.time()
svr.fit(train_X,train_y)

In [ ]:
#show the result of prediction of model
pred_svm_test = svr.predict(test_X)
pred_svm_train = svr.predict(train_X)
svmscore = accuracy_score(test_y,pred_svm_test)
svmscore_train = accuracy_score(train_y,pred_svm_train)
print("mean accuracy score for train set of SVM model is:")
print(svmscore_train)
print("mean accuracy score for test set of SVM model is:")
print(svmscore)
print("confusion matrix for train set is:")
draw_confusion(train_y,pred_svm_train,"Confusion Matrix of train set")
print("confusion matrix for test set is:")
draw_confusion(test_y, pred_svm,"Confusion Matrix for test set")
print('Time taken :')
print( time.time()-t0)

In [ ]:
print("parameter setting is:")
print(svr)

# **Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
parameters = {'max_depth':[10, 20,30], 'splitter':('random','best')}
clf_dt = GridSearchCV(dt, parameters,cv = 5)
# Fit the random search model
clf_dt.fit(train_X, train_y)
print('Best parameter is:')
print(clf_dt.best_params_)
plot_grid_searchlr(clf_dt.cv_results_, clf_dt.cv_results_['params'], 'N Estimators')

md = clf_dt.best_params_['max_depth']
sp = clf_dt.best_params_['splitter']

clf_dt = DecisionTreeClassifier(max_depth =md,splitter = sp)

t0 = time.time()
clf_dt.fit(train_X,train_y)
pred_dt_train = clf_dt.predict(train_X)
pred_dt_test = clf_dt.predict(test_X)
dtscore_train = accuracy_score(train_y,pred_dt_train)
dtscore_test = accuracy_score(test_y,pred_dt_test)
print(clf_dt)
print("mean accuracy score for train set of Decision tree model is:")
print(dtscore_train)
print("mean accuracy score for test set of Decision tree model is:")
print(dtscore_test)
print("confusion matrix for train set is:")
draw_confusion(train_y,pred_dt_train,"Confusion Matrix of train set")
print("confusion matrix for test set is:")
draw_confusion(test_y, pred_dt_test,"Confusion Matrix for test set")
print('Time taken :')
print( time.time()-t0)
print(clf_dt)


In [ ]:
#reference:https://medium.com/@rnbrown/creating-and-visualizing-decision-trees-with-python-f8e8fa394176
#to plot the decision tree diagram
lst1 = df.columns
lst1 = lst1.drop('RainTomorrow')
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
dot_data = StringIO()
export_graphviz(clf_dt, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

# **Random Forest**

In [ ]:
#reference: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
#to find the best parameter setting
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
parameters = {'max_depth':[10, 20,30], 'n_estimators':[200, 400]}
rf = RandomForestClassifier()
clf_rf = GridSearchCV(rf, parameters,cv = 5)
# Fit the random search model
clf_rf.fit(train_X, train_y)
print('Best parameter is:')
print(clf_rf.best_params_)
plot_grid_searchlr(clf_rf.cv_results_, clf_rf.cv_results_['params'], 'N Estimators')

In [ ]:
#fit the model with the best parameterset chosen
md = clf_rf.best_params_['max_depth']
ne = clf_rf.best_params_['n_estimators']
clf_rf =  RandomForestClassifier(max_depth =md,n_estimators = ne)
t0 = time.time()
clf_rf.fit(train_X,train_y)

In [ ]:
#produce results
pred_rf_test = clf_rf.predict(test_X)
pred_rf_train = clf_rf.predict(train_X)
rfscore = accuracy_score(test_y,pred_rf_test)
rfscore_train = accuracy_score(train_y,pred_rf_train)
print("mean accuracy score for train set of Random Forest model is:")
print(rfscore_train)
print("mean accuracy score for test set of Random Forest model is:")
print(rfscore)
print("confusion matrix for train set is:")
draw_confusion(train_y,pred_rf_train,"Confusion Matrix of train set")
print("confusion matrix for test set is:")
draw_confusion(test_y, pred_rf_test,"Confusion Matrix for test set")
print('Time taken :')
print( time.time()-t0)
print("Parameter setting is")
print(clf_rf)

In [ ]:
#Reference: https://towardsdatascience.com/how-to-visualize-a-decision-tree-from-a-random-forest-in-python-using-scikit-learn-38ad2d75f21c
#to visualize the decision tree procedure
lst1 = df.columns
lst1 = lst1.drop('RainTomorrow')
from sklearn.tree import export_graphviz
# Extract single tree
estimator = clf_rf.estimators_[5]

export_graphviz(estimator, out_file='tree.dot', 
                feature_names = lst1,
                class_names = 'RainTomorrow',
                rounded = True, proportion = False, 
                precision = 2, filled = True)

# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')

# **Ensemble methods**

# *Max Voting*

In [ ]:
#https://blog.statsbot.co/ensemble-learning-d1dcd548e936
#build the VotingClassifier model and output the results
from sklearn.ensemble import VotingClassifier
model = VotingClassifier(estimators=[('lr', clf_sgd), ('nn', mlp), ('dt', clf_dt), ('svm', svr),('rf', clf_rf)], voting='hard')
t0 = time.time()
model.fit(train_X,train_y)
mv_pred_train = model.predict(train_X)
mv_pred_test = model.predict(test_X)
print("mean accuracy score of Max voting ensemble method on train set is")
print(model.score(train_X,train_y))
print("mean accuracy score of Max voting ensemble method on test set is")
print(model.score(test_X,test_y))
print("confusion matrix is:")
draw_confusion(train_y,mv_pred_train,"Confusion Matrix of train set")
draw_confusion(test_y,mv_pred_test,"Confusion Matrix of train set")
print("time taken for this method is:")
print(time.time()-t0)

# **Compare performance with using dataset selected by feature selection**

In [ ]:
#reference:https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html
from sklearn.feature_selection import SelectKBest, chi2
selector = SelectKBest(chi2, k=5)
selector.fit(X, y)
X_new = selector.transform(X)
print("Selected features")
print(X.columns[selector.get_support(indices=True)])

In [ ]:
#print the parameter setting
print("Parameter settings")
print(selector)

In [ ]:
#reduce features into selected features from above
train_X = train_X[['Rainfall','Sunshine','Humidity3pm', 'Cloud3pm', 'RainToday']]
test_X = test_X[['Rainfall','Sunshine','Humidity3pm', 'Cloud3pm', 'RainToday']]
#fit the data again with reduced data
clf_sgd.fit(train_X,train_y)
mlp.fit(train_X,train_y)
clf_dt.fit(train_X,train_y)
svr.fit(train_X,train_y)
clf_rf.fit(train_X,train_y)
#call the MAX Voting classifier
model = VotingClassifier(estimators=[('lr', clf_sgd), ('nn', mlp), ('dt', clf_dt), ('svm', svr),('rf', clf_rf)], voting='hard')
t0 = time.time()
model.fit(train_X,train_y)
mv_pred_train = model.predict(train_X)
mv_pred_test = model.predict(test_X)
print("mean accuracy score of Max voting ensemble method on train set is")
print(model.score(train_X,train_y))
print("mean accuracy score of Max voting ensemble method on test set is")
print(model.score(test_X,test_y))
print("confusion matrix is:")
draw_confusion(train_y,mv_pred_train,"Confusion Matrix of train set")
draw_confusion(test_y,mv_pred_test,"Confusion Matrix of train set")
print("time taken for this method is:")
print(time.time()-t0)

In [ ]:
#print the model setting
print(model)